# Nicolas

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
routes = pd.read_csv('data/routes.dat', sep=',', encoding='utf-8')
routes.columns = ['Airline','AirlineID','SourceAirport','SourceAirportID','DestinationAirport','DestinationAirportID','Codeshare','Stops','Equipment']
routes.head()

,Airline,AirlineID,SourceAirport,SourceAirportID,DestinationAirport,DestinationAirportID,Codeshare,Stops,Equipment
0,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
2,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
3,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
4,2B,410,DME,4029,KZN,2990,NaN,0,CR2


In [4]:
airlines = pd.read_csv('data/airlines.dat', sep=',', encoding='utf-8')
airlines.columns = ['Airline ID', 'Name', 'Alias',  'IATA', 'ICAO','Callsign','Country','Active']
airlines.head()

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,1,Private flight,\N,-,NaN,NaN,NaN,Y
1,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N
4,5,213 Flight Unit,\N,NaN,TFU,NaN,Russia,N


In [5]:
airports = pd.read_csv('data/airports.dat', sep=',', encoding='utf-8')
airports.columns = ['Airport ID', 'Name', 'City', 'Country',  'IATA', 'ICAO','Latitude','Longitude','Altitude', 'Timezone','DST', 'Tz database time zone', 'Type', 'Source']
airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
3,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
4,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports


In [42]:
alliances = pd.read_csv('data/alliances.dat', sep=',', encoding='utf-8')
alliances.head()

,Alliance,Airline
0,Star Alliance,Adria Airways
1,Star Alliance,Aegean Airlines
2,Star Alliance,Air Canada
3,Star Alliance,Air China
4,Star Alliance,Air New Zealand


In [43]:
alliances.shape

(60, 2)

In [44]:
merged  = pd.merge(alliances, airlines, left_on='Airline', right_on='Name', how = 'left')
merged[merged['Airline ID'].isnull()]

,Alliance,Airline,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active


In [45]:
merged

,Alliance,Airline,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,Star Alliance,Adria Airways,83,Adria Airways,\N,JP,ADR,ADRIA,Slovenia,Y
1,Star Alliance,Aegean Airlines,96,Aegean Airlines,\N,A3,AEE,AEGEAN,Greece,Y
2,Star Alliance,Air Canada,330,Air Canada,\N,AC,ACA,AIR CANADA,Canada,Y
3,Star Alliance,Air China,751,Air China,\N,CA,CCA,AIR CHINA,China,Y
4,Star Alliance,Air New Zealand,345,Air New Zealand,\N,NZ,ANZ,NEW ZEALAND,New Zealand,Y
5,Star Alliance,All Nippon Airways,324,All Nippon Airways,ANA All Nippon Airways,NH,ANA,ALL NIPPON,Japan,Y
6,Star Alliance,Asiana Airlines,28,Asiana Airlines,\N,OZ,AAR,ASIANA,Republic of Korea,Y
7,Star Alliance,Austrian Airlines,491,Austrian Airlines,\N,OS,AUA,AUSTRIAN,Austria,Y
8,Star Alliance,Avianca - Aerovias Nacionales de Colombia,515,Avianca - Aerovias Nacionales de Colombia,\N,AV,AVA,S.A.,AVIANCA,Y
9,Star Alliance,Brussels Airlines,1531,Brussels Airlines,SN Brussels Airlines,SN,DAT,BEE-LINE,Belgium,Y
